In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
from Project_3_5 import solve_pfr_pyrolysis

# ==========================================================
# MECHANISM + GENERAL INPUTS
# ==========================================================
mech = "cai_pisch.yaml"
X0 = {"CH4":1.0}
T0 = 300.0
p0 = ct.one_atm
x_end = 5.0      # reactor length (can modify — just must be fixed for all runs)
Npts = 600

# Species to track
tracked_species = ["CH4", "H2", "C2H2", "C2H4"]

# Helper function → returns final values
def run_case(Tw, u0, D):
    """Runs pyrolysis case + returns exit values + full solution"""

    sol = solve_pfr_pyrolysis(mech, X0, T0, p0, u0, D, Tw, x_end, Npts=Npts)
    x = sol["x"]; T = sol["T"]; X = sol["X"]; gas = sol["gas"]; mdot = sol["mdot"]

    # extract species
    sp = {}
    for s in tracked_species:
        idx = gas.species_index(s) if s in gas.species_names else None
        sp[s] = X[idx,-1] if idx is not None else np.nan

    # heat input
    gas.TPX = T0, p0, X0
    Q = mdot * gas.cp_mass * (Tw - T0)

    # efficiency
    eff = (T[-1]-T0)/(Tw-T0)

    return T[-1], sp, Q, eff


# ==========================================================
# STUDY 1 — Sweep Wall Temperature
# ==========================================================
Tw_range = np.linspace(1100,2000,10)
u_nom = 1.0         # m/s
D_nom = 0.0254      # 1 inch

T_tw=[]; Q_tw=[]; eff_tw=[]
CH4_tw=[]; H2_tw=[]; C2H2_tw=[]; C2H4_tw=[]

for Tw in Tw_range:
    T_end, sp, Q, eff = run_case(Tw,u_nom,D_nom)
    T_tw.append(T_end); Q_tw.append(Q); eff_tw.append(eff)
    CH4_tw.append(sp["CH4"]); H2_tw.append(sp["H2"])
    C2H2_tw.append(sp["C2H2"]); C2H4_tw.append(sp["C2H4"])

# --- PLOTS (1–4) ---
plt.figure(); plt.plot(Tw_range,T_tw); plt.title("Temperature vs Wall Temperature"); plt.xlabel("Tw (K)"); plt.ylabel("Exit T (K)"); plt.grid()
plt.figure(); plt.plot(Tw_range,CH4_tw,label="CH4"); plt.plot(Tw_range,H2_tw,label="H2"); plt.plot(Tw_range,C2H2_tw,label="C2H2"); plt.plot(Tw_range,C2H4_tw,label="C2H4"); plt.legend(); plt.title("Mole Fractions vs Wall Temperature"); plt.xlabel("Tw (K)"); plt.ylabel("Mole Fraction"); plt.grid()
plt.figure(); plt.plot(Tw_range,Q_tw); plt.title("Heat Input vs Wall Temperature"); plt.xlabel("Tw (K)"); plt.ylabel("Q (W)"); plt.grid()
plt.figure(); plt.plot(Tw_range,eff_tw); plt.title("Efficiency vs Wall Temperature"); plt.xlabel("Tw (K)"); plt.ylabel("η"); plt.grid()


# ==========================================================
# STUDY 2 — Sweep VELOCITY
# ==========================================================
u_range = np.logspace(-1,1,10)
Tw_nom = 1500
D_nom = 0.0254

T_u=[]; Q_u=[]; eff_u=[]
CH4_u=[]; H2_u=[]; C2H2_u=[]; C2H4_u=[]

for u in u_range:
    T_end, sp, Q, eff = run_case(Tw_nom,u,D_nom)
    T_u.append(T_end); Q_u.append(Q); eff_u.append(eff)
    CH4_u.append(sp["CH4"]); H2_u.append(sp["H2"])
    C2H2_u.append(sp["C2H2"]); C2H4_u.append(sp["C2H4"])

# --- PLOTS (5–8) ---
plt.figure(); plt.semilogx(u_range,T_u); plt.title("Temperature vs Flow Velocity"); plt.xlabel("u (m/s)"); plt.ylabel("Exit T (K)"); plt.grid()
plt.figure(); plt.semilogx(u_range,CH4_u,label="CH4"); plt.semilogx(u_range,H2_u,label="H2"); plt.semilogx(u_range,C2H2_u,label="C2H2"); plt.semilogx(u_range,C2H4_u,label="C2H4"); plt.legend(); plt.title("Mole Fractions vs Velocity"); plt.xlabel("u (m/s)"); plt.ylabel("Mole Fraction"); plt.grid()
plt.figure(); plt.semilogx(u_range,Q_u); plt.title("Heat Input vs Velocity"); plt.xlabel("u (m/s)"); plt.ylabel("Q (W)"); plt.grid()
plt.figure(); plt.semilogx(u_range,eff_u); plt.title("Efficiency vs Velocity"); plt.xlabel("u (m/s)"); plt.ylabel("η"); plt.grid()


# ==========================================================
# STUDY 3 — Sweep DIAMETER
# ==========================================================
D_range = np.linspace(0.00254,0.0508,10)  # 0.1–2 in
Tw_nom = 1500
u_nom  = 1.0

T_d=[]; Q_d=[]; eff_d=[]
CH4_d=[]; H2_d=[]; C2H2_d=[]; C2H4_d=[]

for D in D_range:
    T_end, sp, Q, eff = run_case(Tw_nom,u_nom,D)
    T_d.append(T_end); Q_d.append(Q); eff_d.append(eff)
    CH4_d.append(sp["CH4"]); H2_d.append(sp["H2"])
    C2H2_d.append(sp["C2H2"]); C2H4_d.append(sp["C2H4"])

# --- PLOTS (9–12) ---
plt.figure(); plt.plot(D_range,T_d); plt.title("Temperature vs Diameter"); plt.xlabel("D (m)"); plt.ylabel("Exit T (K)"); plt.grid()
plt.figure(); plt.plot(D_range,CH4_d,label="CH4"); plt.plot(D_range,H2_d,label="H2"); plt.plot(D_range,C2H2_d,label="C2H2"); plt.plot(D_range,C2H4_d,label="C2H4"); plt.legend(); plt.title("Mole Fractions vs Diameter"); plt.xlabel("D (m)"); plt.ylabel("Mole Fraction"); plt.grid()
plt.figure(); plt.plot(D_range,Q_d); plt.title("Heat Input vs Diameter"); plt.xlabel("D (m)"); plt.ylabel("Q (W)"); plt.grid()
plt.figure(); plt.plot(D_range,eff_d); plt.title("Efficiency vs Diameter"); plt.xlabel("D (m)"); plt.ylabel("η"); plt.grid()

plt.show()